In [4]:
import random
import csv
from typing import List
import hashlib
import uuid
from datetime import datetime
import pandas as pd


class CSVInitiliser:
    def __init__(self, filename, filepath='') -> None:
        self.filename = filename
        self.filepath = filepath

    def create_file(self, header):
        try:
            with open(self.filepath + self.filename, 'x', encoding='utf-8-sig',  newline='') as file:
                writer = csv.writer(file, delimiter=',',
                                    quotechar=',', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(header)

        except FileExistsError:
            pass

    def write_row(self, row):
        with open(self.filepath + self.filename, 'a') as file:
            writer = csv.writer(file)
            writer.writerow(row)

    def get_dataframe(self):
        return pd.read_csv(self.filepath + self.filename, keep_default_na=False)


class Account:
    def __init__(self, p_uid: str, initial_balance: float = 0, filepath: str = ''):
        self.p_uid = p_uid
        self.filepath = filepath
        self.balance = initial_balance
        self.init_csv()
        self.init_person()

    def init_csv(self):
        self.output_file = CSVInitiliser(
            'transaction_history.csv', self.filepath)
        self.output_file.create_file(
            ['person_uid', 'opeation_uid', 'operation_category', 'money', 'created_at'])

    def init_person(self):
        df = self.output_file.get_dataframe()
        df = df[df.person_uid == self.p_uid]
        self.transaction_history = []
        if len(df):
            df = df.reset_index(drop=True)
            self.balance = sum(df.money)
            self.transaction_history = [
                f'{df.iloc[idx].operation_category}: {df.iloc[idx].money} {df.iloc[idx].created_at}' for idx in range(len(df))]

    def deposit(self, amount: float):
        if amount <= 0:
            raise ValueError("Сумма депозита должна быть положительной.")

        now = datetime.now()
        self.balance += amount
        self.transaction_history.append(f'Депозит: {amount} {now}')
        self.output_file.write_row(
            [self.p_uid, str(uuid.uuid4()), 'Депозит', amount, now])

    def withdraw(self, amount: float):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной.")

        if amount > self.balance:
            raise ValueError("Недостаточно средств на счёте.")

        now = datetime.now()
        self.balance -= amount
        self.transaction_history.append(f'Снятие: -{amount} {now}')
        self.output_file.write_row(
            [self.p_uid, str(uuid.uuid4()), 'Снятие', -amount, now])

    def get_balance(self) -> float:
        return self.balance

    def get_transaction_history(self) -> List[str]:
        return '\n'.join(self.transaction_history)


class Person:
    def __init__(self, name: str, password: str = '', mail: str = '', uid='', phash=''):
        self.id = str(uuid.uuid4()) if not uid else uid
        self.name = name
        self.mail = mail
        self.phash = phash
        self.set_password(password)

    def set_password(self, password: str):
        if not password and not self.phash:
            raise ValueError("Пароль должен быть непустым")
        self.password_hash = self.hash_password(
            password) if not self.phash else self.phash

    def hash_password(self, password: str) -> str:
        return hashlib.sha256(password.encode('utf-8')).hexdigest()

    def check_password(self, password: str) -> bool:
        return self.hash_password(password) == self.password_hash

    def __str__(self):
        return f"ID: {self.id}, ФИО: {self.name}"


class PersonManager:
    def __init__(self, filepath=''):
        self.filepath = filepath
        self.init_csv()
        self.init_people()

    def init_csv(self):
        self.output_file = CSVInitiliser('person_list.csv', self.filepath)
        self.output_file.create_file(
            ['person_uid', 'name', 'mail', 'password_hash', 'created_at'])

    def init_people(self):
        df = self.output_file.get_dataframe()
        self.people = []
        if len(df):
            self.people = [Person(name=df.iloc[idx].name, uid=df.iloc[idx].person_uid, mail=df.iloc[idx].mail,
                                  phash=df.iloc[idx].password_hash) for idx in range(len(df))]

    def add_person(self, name: str, password: str, mail: str) -> Person:
        df = self.output_file.get_dataframe()
        current_df = df[df.mail == mail].drop_duplicates()
        if not len(df[df.mail == mail]):
            person = Person(name, password, mail)
            self.people.append(person)
            self.output_file.write_row(
                [person.id, person.name, person.mail, person.password_hash, datetime.now()])
            return person
        else:
            current_person = current_df.sort_values('created_at')
            return Person(name=name, uid=current_person.iloc[0].person_uid,
                          phash=current_person.iloc[0].password_hash, mail=mail)

    def find_person_by_id(self, person_id: str) -> Person:
        for person in self.people:
            if person.id == person_id:
                return person
        return None

    def authenticate(self, person_id: str, password: str) -> bool:
        person = self.find_person_by_id(person_id)
        if person:
            return person.check_password(password)
        return False


class Bank:
    def __init__(self, name: str, filepath: str = ''):
        self.name = name
        self.id = str(uuid.uuid4())
        self.filepath = filepath
        self.init_csv()
        self.init_people()

    def init_csv(self):
        self.output_file = CSVInitiliser('bank_list.csv', self.filepath)
        self.output_file.create_file(
            ['bank_uuid', 'bank_name', 'person_uuid', 'created_at'])

    def init_people(self):
        df = self.output_file.get_dataframe()
        df = df[df.bank_name == self.name].reset_index(drop=True)
        self.users = []
        if len(df):
            self.users = [df.iloc[idx].person_uuid for idx in range(len(df))]

    def add_user(self, user_id) -> Person:
        if not self.find_user_by_id(user_id):
            self.users.append(user_id)
            self.output_file.write_row(
                [self.id, self.name, user_id, datetime.now()])

    def get_users(self) -> list:
        return self.users

    def find_user_by_id(self, user_id: str) -> Person:
        for uid in self.users:
            if uid == user_id:
                return uid
        return None

    def __str__(self):
        return f"Навзание банка: {self.name}, Кол-во пользователей: {len(self.users)}"


if __name__ == "__main__":
    tbank = Bank("TBank")
    sbank = Bank("SBank")
    p_manger = PersonManager()
    p1 = p_manger.add_person("Alexey Alexeyvich", "qwerty123", 'AlexeyAlexeyvich@gmail.com')
    p2 = p_manger.add_person("Mikhail Dmitrievich", "312dsadas", 'MikhailDmitrievich@yandex.ru')
    if p_manger.authenticate(p1.id, 'qwerty123'): 
        print(p1)
        tbank.add_user(p1.id)
        print(tbank)
        p1_acc = Account(p1.id)
        p1_acc.deposit(random.randint(10 * 1000, 100 * 1000))
        p1_acc.withdraw(random.randint(10 * 1000, 100 * 1000))
        print(p1_acc.get_transaction_history())
        print(p1_acc.balance)

ID: 83946049-97a4-4701-8264-505df2f5de7b, ФИО: Alexey Alexeyvich
Навзание банка: TBank, Кол-во пользователей: 1
Депозит: 43057 2024-09-10 09:19:09.348752
Снятие: -26114 2024-09-10 09:19:09.348976
Депозит: 60416 2024-09-10 09:19:11.905098
Депозит: 48422 2024-09-10 09:19:17.268434
Снятие: -73419 2024-09-10 09:19:17.269899
Депозит: 21224 2024-09-10 09:19:22.878081
Снятие: -54020 2024-09-10 09:19:22.878377
19566
